## Getting all entries from DB

In [27]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('synthesis.db')

# SQL query to fetch all rows from the syntree table
query = """
SELECT * FROM syntree;
"""

# Execute the query and load results into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()



In [28]:
df.shape

(185366, 10)

In [30]:
(df.path.str.lower() != df.search_path.str.lower()).sum()


0

In [8]:
# Learning 1. path and search path are identifcal!!

In [31]:
# Split 'path' column on the first ' - ' occurrence, assigning first part to 'category' and the rest to 'Relevantes Symptom'
df[['category', 'Relevantes Symptom']] = df['path'].str.split(' - ', n=1, expand=True)

In [94]:
df.category.value_counts()

category
Extremitäten                 28136
Gemüt                        19240
Kopf                         13864
Allgemeines                  12707
Abdomen                       9428
Brust                         9061
Rücken                        8478
Magen                         7619
Gesicht                       5345
Mund                          4846
Auge                          4798
Weibliche Genitalien          4505
Träume                        4155
Ohr                           3664
Nase                          3501
Rektum                        3453
Haut                          3207
Schlaf                        2893
Innerer Hals                  2787
Husten                        2604
Männliche Genitalien          2593
Atmung                        2413
Zähne                         2340
Sehen                         1794
Blase                         1663
Kehlkopf und Trachea          1491
Fieber                        1393
Frost                         1278
Schwindel  

In [38]:
# Remove Veterinät for now:
df.shape

(185366, 12)

In [41]:
df = df[df.category != "Veterinär"]

- we leave out "Veterinär" for now

In [44]:
df.category.nunique()

41

In [46]:
df["Relevantes Symptom"].isnull().sum() # weil jede kategorie erstmal auf nichts folgt mit " - "

41

In [61]:
df.shape[0] - 41

176489

In [62]:
df_no_null = df[~df["Relevantes Symptom"].isnull()]

In [63]:
df_no_null

,id,id_father,item,image,ref_id,level,chapter_id,order_index,path,search_path,category,Relevantes Symptom
1,9183339,9183338.0,Tagsüber,None,4.0,2,9183338.0,1,Gemüt - Tagsüber,GEMÜT - TAGSÜBER,Gemüt,Tagsüber
2,9183340,9183338.0,Morgens,None,5.0,2,9183338.0,2,Gemüt - Morgens,GEMÜT - MORGENS,Gemüt,Morgens
3,9183341,9183340.0,abends; und,None,6.0,3,9183338.0,3,Gemüt - Morgens - abends; und,GEMÜT - MORGENS - ABENDS; UND,Gemüt,Morgens - abends; und
4,9183342,9183338.0,Vormittags,None,7.0,2,9183338.0,4,Gemüt - Vormittags,GEMÜT - VORMITTAGS,Gemüt,Vormittags
5,9183343,9183338.0,Mittags,None,8.0,2,9183338.0,5,Gemüt - Mittags,GEMÜT - MITTAGS,Gemüt,Mittags
...,...,...,...,...,...,...,...,...,...,...,...,...
176525,9359863,9359858.0,Fieber; während,None,170414.0,3,9347161.0,176525,Allgemeines - Zyanose - Fieber; während,ALLGEMEINES - ZYANOSE - FIEBER; WÄHREND,Allgemeines,Zyanose - Fieber; während
176526,9359864,9359858.0,Kindern; bei,None,170411.0,3,9347161.0,176526,Allgemeines - Zyanose - Kindern; bei,ALLGEMEINES - ZYANOSE - KINDERN; BEI,Allgemeines,Zyanose - Kindern; bei
176527,9359865,9359864.0,Geburt an; von der,None,170412.0,4,9347161.0,176527,Allgemeines - Zyanose - Kindern; bei - Geburt ...,ALLGEMEINES - ZYANOSE - KINDERN; BEI - GEBURT ...,Allgemeines,Zyanose - Kindern; bei - Geburt an; von der
176528,9359866,9359864.0,Kleinkindern; bei,None,170413.0,4,9347161.0,176528,Allgemeines - Zyanose - Kindern; bei - Kleinki...,ALLGEMEINES - ZYANOSE - KINDERN; BEI - KLEINKI...,Allgemeines,Zyanose - Kindern; bei - Kleinkindern; bei


- Decision: I decide to only emebdd "relevanted Symptom", but could well be that category + relevantes symptom" makes sense.
- we go for "relevantes Symptom" first

# Start EMebdding job

In [64]:
def get_secret():
    secret_name = "openai/api-key/chat-app-keywords"
    region_name = "eu-central-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    print(json.loads(secret)["OPENAI_KEY"])
    return json.loads(secret)["OPENAI_KEY"]

In [65]:
import openai
import boto3
import json
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = get_secret()
client = OpenAI()

sk-proj-ybXFa9nUWWZy0FvTaqcnT3BlbkFJO5EFVKxJJc07xyc8qyVD


In [1]:
import pandas as pd


In [69]:
data = df_no_null.copy()

In [70]:
data.shape

(176489, 12)

In [71]:
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm

In [72]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embeddings(texts, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=texts, model=model)
    return [item.embedding for item in response.data]


In [75]:
# data["Relevantes Symptom"]

In [76]:
batch_size = 100  # Adjust based on your rate limit and requirements
embeddings = []

# Initialize tqdm progress bar
for i in tqdm(range(0, len(data), batch_size), desc="Processing Batches"):
    batch_texts = data['Relevantes Symptom'].iloc[i:i + batch_size].tolist()
    batch_embeddings = get_embeddings(batch_texts)
    embeddings.extend(batch_embeddings)

data['relevant_symptom_embeddings'] = embeddings



Processing Batches: 100%|███████████████████| 1765/1765 [41:50<00:00,  1.42s/it]


In [83]:
# df

In [86]:
# data.to_parquet("relevant_symptoms.gz")

In [89]:
# pd.read_parquet("relevant_symptoms.gz")

In [93]:
# data.category.unique()

In [59]:
np.vstack(data['embeddings'].values).shape

(5000, 1536)

In [11]:
def search_top_similar_symptoms(user_query, data, top_n=10):
    # Schritt 1: Benutzereingabe entgegennehmen und bereinigen (optional)
    # Hier nehmen wir den Text direkt

    # Schritt 2: Embedding für die Benutzereingabe generieren
    user_embedding = get_embeddings([user_query])[0]  # Annahme: get_embeddings gibt eine Liste von Embeddings zurück

    # Schritt 3: Vorbereitung der gespeicherten Embeddings
    # Konvertiere die Embeddings-Spalte zu einem 2D-NumPy-Array, falls noch nicht geschehen
    # Angenommen, 'embeddings' sind als Listen gespeichert
    embeddings_matrix = np.vstack(data['embeddings'].values)

    # Schritt 4: Berechnung der Kosinusähnlichkeit ohne Normalisierung
    cosine_similarities = np.dot(embeddings_matrix, user_embedding)

    # Schritt 5: Sortierung und Auswahl der Top-N ähnlichen Einträge
    top_indices = cosine_similarities.argsort()[::-1][:top_n]
    top_scores = cosine_similarities[top_indices]
    top_symptoms = data.iloc[top_indices].copy()
    top_symptoms['similarity'] = top_scores


    # Schritt 7: Präsentation der Ergebnisse
    return top_symptoms


In [24]:
# Beispielaufruf:
user_input = "Beten, Nacht"
top_results = search_top_similar_symptoms(user_input, data, top_n=5)

In [25]:
top_results

,symptom_id,origin_text,search_path,relevant_text,embeddings,similarity
1830,9185168,GEMÜT - BETEN - NACHTS,GEMÜT,BETEN - NACHTS,"[-0.03742961585521698, 0.01724584586918354, -0...",0.634370
375,9183713,GEMÜT - ANGST - NACHTS - BETT; IM,GEMÜT,ANGST - NACHTS - BETT; IM,"[-0.021743617951869965, 0.012017698958516121, ...",0.580540
3314,9186653,"GEMÜT - ERREGUNG - ABENDS - BETT, IM",GEMÜT,"ERREGUNG - ABENDS - BETT, IM","[0.020391957834362984, 0.026025382801890373, -...",0.575588
1342,9184680,GEMÜT - BEIßEN - NACHTS,GEMÜT,BEIßEN - NACHTS,"[-0.030466852709650993, 0.02105293795466423, -...",0.573611
105,9183443,GEMÜT - ABNEIGUNG - NACHT; GEGEN DIE,GEMÜT,ABNEIGUNG - NACHT; GEGEN DIE,"[-0.006598934531211853, 0.05377455800771713, -...",0.551492


In [80]:
import os
from langchain_openai import ChatOpenAI

In [16]:
system_message = "Du bist deutscher Homöopath und analysierst die Symptome des Patienten nach dem Buch des Synthesis."

In [22]:
import os
from openai import OpenAI

# Initialisieren des OpenAI-Clients
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def enrich_query(user_input, sys_mes):
    """
    Verwendet GPT-4o, um die Hauptsymptome aus der Benutzereingabe zu extrahieren und zu erweitern.
    """
    messages = [
        {"role": "system", "content": f"{sys_mes}"},
        {"role": "user", "content": f"Extrahiere die Symptome des Patienten und paraphrasiere die Symptomatik falls zutreffend nach dem deutschen Synthesis Buch der Homöopathen. Bitte die Schlüssel-Symptome nach Synthesis extrahieren, gerne Komma separiert. Bitte NUR die Symptomatik geben, kurz und präzise.\n\nUser Input: {user_input}\n\n"},
        {"role": "user", "content": f"Bitte antworte als würdest du versuchen so präzise wie möglich den Suchpfad im Synthesis zu finden, auf Basis der Symptomatik."},
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=30,
        temperature=0.1
    )
    
    enriched_query = response.choices[0].message.content.strip()
    return enriched_query

# Beispielhafte Verwendung
user_input = "Der Patient betet in der Nacht"
enriched_query = enrich_query(user_input, system_message)
print("Erweiterte Anfrage:", enriched_query)


Erweiterte Anfrage: Beten, Nacht, Schlaf, Schlaflosigkeit, religiöse Gedanken.
